# GENETIC ALGORITHM FOR BIN PACKING

## Imports and data injestion

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
from math import floor
from random import randint
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
import scipy 
from scipy.misc import comb # comb(n,k, exact=True)
# from scipy import special
# from scipy.special import comb



CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [2]:
items = 'items'
bins = 'bins'
items_2D = 'items2D'
bins_2D = 'bins2D'

def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns

### Item data

In [3]:
#ONE DIMENSIONAL
df_items1 = read_data('%s.csv'%items)
df_items1.set_index('item')
#TWO DIMENSIONAL
df_items2 = read_data('%s.csv'%items_2D)
df_items2.set_index('item')
display(df_items1)
display(df_items2)

,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,NaN
3,3,3,NaN
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


,item,x_size,y_size,bin
0,0,4,2,NaN
1,1,3,1,NaN
2,2,6,8,NaN
3,3,3,2,NaN
4,4,2,1,NaN
5,5,1,1,NaN
6,6,2,2,NaN


### Bin Data

In [4]:
#ONE DIMENSIONAL
df_vehicles1 = read_data('%s.csv'%bins)
df_vehicles1.set_index('bin')
#TWO DIMENSIONAL
df_vehicles2 = read_data('%s.csv'%bins_2D)
df_vehicles2.set_index('bin')
display(df_vehicles1)
display(df_vehicles2)

,bin,size,available_space
0,0,9,9
1,1,3,3
2,2,4,4


,bin,x_size,y_size,av_x_space,av_y_space
0,0,9,6,9,6
1,1,5,3,5,3
2,2,7,5,7,5


# Group Genetic Algorithm (GGA) for Bin Packing Problem (BPP)

##### From: Paper[GGA_FALKENEUR]
Group Genetic Algorithm (GGA), is a modification to the genetic algorithm to suit the structure of grouping problems. We aim to find a good partition of a set, or to group together the elements of the set. 
4 Steps:
1. ENCODING: obtain initial population for each bin, by randomising the order of the items and applying the first ft algorithm. All the solutions are parents.
2. CROSSOVER: Has 5 sub-steps: 
    1. Select the same random crossing site in two parents.
    2. Injest the contents of the first parent at the cross section in at the second parent's cross section, creating children.
    3. Replicated elements are removed.  
    4. Adapt results such that constraints are adhered to. 
    5. Rank the fitness of the children.
3. MUTATION: Shuffle a small group of items among groups.
4. INVERSION: do a random bit swop.

NOTE: IT IS AN UNEDUATED SEARCH SPACE CREATED

## STEP 1: ENCODING
1. Generating the initial population for th $k$ bins. For $n$ items, generate $n$ parents. 
2. Apply first-fit algorithm to each of the $n$ lists. 
3. For each list, we generate a solution for each of the $k$ bins.
4. TABLE: $x$ = LIST$_i$ Bins, and $y$ = Items, filled with bin j accordingly. Where $i \epsilon n$, and $j \epsilon k$  
5. OUTPUT: A population of bins to which items are allocated
6. <font color='red'>NOTE: LISTS = THE VARIOUS COMBINATIONS OF FIRST FIT ALGORITHM APPLIED

### First-fit algorithm for 1 and 2 dimensional BPP

In [5]:
### ONE DIMENSIONAL FIRST_FIT ALGORITHM##
def first_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.iat[i,1]
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.iat[j,2]
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,2] = df_vehicles.iat[j,2] - item_size #update avialable space
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,2] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,2] = np.nan
    return df_items, df_vehicles

### TWO DIMENSIONAL FIRST_FIT ALGORITHM##
def first_fit_2D(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size_x = df_items.iat[i,1]
        item_size_y = df_items.iat[i,2]
        while j<= number_of_bins and item_allocated == False:
            available_bin_space_x = df_vehicles.iat[j,3]
            available_bin_space_y = df_vehicles.iat[j,4]
            if available_bin_space_x >= item_size_x and available_bin_space_y >= item_size_y: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,3] = df_vehicles.iat[j,3] - item_size_x #update avialable space in x direction
                df_vehicles.iat[j,4] = df_vehicles.iat[j,4] - item_size_y #update avialable space in y direction 
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,3] = bin_num#set the allocated bin for the item
            elif available_bin_space_x >= item_size_y and available_bin_space_y >= item_size_x: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,3] = df_vehicles.iat[j,3] - item_size_y #update avialable space in x direction
                df_vehicles.iat[j,4] = df_vehicles.iat[j,4] - item_size_x #update avialable space in y direction 
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,3] = bin_num#set the allocated bin for the item
            elif available_bin_space_x < item_size_x or available_bin_space_y < item_size_y: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,3] = np.nan
    return df_items, df_vehicles

### Encoding population: Initial population
<font color='red'>NOTE: $n!$ options, but have created $n$ samples in the population. FOLLOW UP , as if $n!$ samples then would it not be brute force??

In [6]:
#creates a list of new column names
def rename_col(num_cols):
    string = 'list'
    s1 = []
    for i in range(num_cols): # This is just to tell you how to create a list.
        updated_name = string + str(i)
        s1.append(updated_name)
    return s1


def encoding(df_items, df_vehicles, dimension):
    #INPUT: vehicle dataframe - vehicles and its capacity
    #        item dataframe - item size
    #        dimension - 1 if 1D items/vehicles and 2 if 2D items/vehicles
    n = df_items.shape[0] #number of items
    k = df_vehicles.shape[0]#number of bins
    updated_name = rename_col(n)
    newDF = pd.DataFrame() #initialise a new dataframe
    if dimension == 1:
        newDF = newDF.append(df_items[:][['item','size']])#standard to all is the items with their respective sizes
        #use 1d first fit
        for i in range(n):
            df_items_copy = df_items.copy()# make a copy of the item list
            df_vehicles_copy = df_vehicles.copy()# make a copy of the bin list
            df_items_copy_temp = shuffle(df_items_copy) #shuffle the copy of items
            df_items_copy_temp, df_vehicles_copy = first_fit(df_items_copy_temp,df_vehicles_copy) #assigns items to bins using ff
            df_items_copy_temp = df_items_copy_temp.sort_values(by = 'item').reset_index(drop=True) #rearranges the list according to item 0,1,2... and resets the index as well
            newDF = pd.merge(newDF, df_items_copy_temp, on = ['item', 'size'])#appends the column with the respective bins to which the items has been allocated 
            newDF = newDF.rename(columns = {newDF.columns[-1]: updated_name[i]})# renames the column 
    elif dimension == 2:
        newDF = newDF.append(df_items[:][['item','x_size','y_size']])
        #use 2d first fit
        for i in range(n): 
            df_items_copy = df_items.copy()# make a copy of the item list
            df_vehicles_copy = df_vehicles.copy()# make a copy of the bin list
            df_items_copy_temp = shuffle(df_items_copy) #shuffle the copy of items
            df_items_copy_temp, df_vehicles_copy = first_fit_2D(df_items_copy_temp,df_vehicles_copy) #assigns items to bins using ff 
            df_items_copy_temp = df_items_copy_temp.sort_values(by = 'item').reset_index(drop=True) #rearranges the list according to item 0,1,2... and resets the index as well
            newDF = pd.merge(newDF, df_items_copy_temp, on = ['item', 'x_size', 'y_size'])#appends the column with the respective bins to which the items has been allocated 
            newDF = newDF.rename(columns = {newDF.columns[-1]: updated_name[i]})# renames the column 
    return newDF

In [7]:
population = encoding(df_items1, df_vehicles1,1)
population

,item,size,list0,list1,list2,list3,list4,list5,list6
0,0,4,0.0,NaN,2.0,0.0,2.0,NaN,2.0
1,1,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,6,NaN,0.0,NaN,NaN,NaN,0.0,NaN
3,3,3,0.0,NaN,0.0,1.0,1.0,2.0,0.0
4,4,2,0.0,1.0,1.0,2.0,0.0,0.0,0.0
5,5,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0
6,6,2,2.0,2.0,0.0,2.0,0.0,NaN,1.0


In [8]:
population_2D = encoding(df_items2, df_vehicles2,2) 
population_2D

,item,x_size,y_size,list0,list1,list2,list3,list4,list5,list6
0,0,4,2,1.0,2.0,1.0,0.0,1.0,2.0,1.0
1,1,3,1,0.0,0.0,2.0,1.0,2.0,0.0,2.0
2,2,6,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,2,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,4,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,2,2,2.0,1.0,0.0,0.0,0.0,1.0,0.0


In [9]:
population_2D.drop(['x_size','y_size'], axis=1, inplace=True)
population_2D

,item,list0,list1,list2,list3,list4,list5,list6
0,0,1.0,2.0,1.0,0.0,1.0,2.0,1.0
1,1,0.0,0.0,2.0,1.0,2.0,0.0,2.0
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,2.0,1.0,0.0,0.0,0.0,1.0,0.0


## CROSSOVER: 
### Parent set per bin
Creating a dataframe per bin with parents. Boolean representation if item is in that bin or not per parent. TABLE: X= Parents/lists, Y = items. Filled with true and false. creates childer $(2* ^{parents}C_2)$.
STEPS: 
1. Get the number of bins (vehicles)
2. For each bin create a parent dataframe. ie the initial population in boolean representation of that particular bin.
3. Create children dataframe. The result of crossover of the parents. 

<font color='red'>NOTE: The crossover is happening per bin.

In [10]:
#for a particular bin, creates a boolean representation of the bin for all the lists 
def parents(df_population, bin_number):
    df_population_copy = df_population.copy()
    pop_rows = df_population.shape[0]
    pop_cols = df_population.shape[1]-2
    num_items  = pop_rows #number of items
    newDF = pd.DataFrame(np.zeros((pop_rows,pop_cols))) #initialise a new dataframe
    for j in range(num_items):#iterates all the lists
        #get the columns in which the item is filled with the sepecific item
        list_num = 'list'+str(j)
        temp = df_population_copy.loc[df_population_copy['%s'%(list_num)]==bin_number]
        #get the item values in array format
        temp1 = temp.loc[:,'%s'%('item')].values
        for k in range(len(temp1)):
            #boolean fill the new dataframe
            item_val = temp1[k]
            newDF.loc[item_val][j] = True
    return newDF
                

In [11]:
#creates a dataframe with only bn info of the population
def parent_bins(df_population):
    #1D scenario:
    df_parent_bins = df_population.iloc[:,2:]
    return df_parent_bins

In [12]:
display(parents(population,0))
display(population)

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,1.0,1.0
5,0.0,0.0,1.0,1.0,1.0,1.0,1.0
6,0.0,0.0,1.0,0.0,1.0,0.0,0.0


,item,size,list0,list1,list2,list3,list4,list5,list6
0,0,4,0.0,NaN,2.0,0.0,2.0,NaN,2.0
1,1,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,6,NaN,0.0,NaN,NaN,NaN,0.0,NaN
3,3,3,0.0,NaN,0.0,1.0,1.0,2.0,0.0
4,4,2,0.0,1.0,1.0,2.0,0.0,0.0,0.0
5,5,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0
6,6,2,2.0,2.0,0.0,2.0,0.0,NaN,1.0


In [13]:
#GLOBAL VALUE
num_item1 = df_items1.shape[0]
num_item2 = df_items2.shape[0]
combination_of_children1 = comb(num_item1,2, exact=True)
combination_of_children2 = comb(num_item2,2, exact=True)

In [14]:
#offsprings of the parents: For each combination of parents, there is two offsprings
#INPUT: parent dataframe
#OUTPUT: children dataframe - the crossover of the parent combinations 
def offsprings(df_parent):
    pointer_children = 0
    num_items= df_parent.shape[0]
    combination_of_children = comb(num_items,2,exact=True)*2#number of combinations. Multiply by two as from 2parents we have two children
    random_cross_section = floor(num_items/2)#randint(0,num_items)
    df_offspring = pd.DataFrame(np.zeros((num_items,combination_of_children)))#initializes a dataframe of size [r,c] = [num_items,num_children(combinations of crossovers)]
    
    for i in range(num_items):
        partA1 = df_parent.iloc[0:random_cross_section,i]#top part of parent 1
        partB1 = df_parent.iloc[random_cross_section:, i]#bottom part of parent 1
        for j in range(i,num_items):
            if i != j:
            #crossover between the two indices producing two children
                partA2 = df_parent.iloc[0:random_cross_section,j]#top part of parent 2
                partB2 = df_parent.iloc[random_cross_section:, j]#Bottom part of parent 2
                df_offspring.iloc[0:random_cross_section,pointer_children] = partA1 #top part of parent 1 
                df_offspring.iloc[random_cross_section:,pointer_children] = partB2 #Bottom part of parent 2
                df_offspring.iloc[0:random_cross_section,pointer_children+1] = partA2 #top part of parent 2
                df_offspring.iloc[random_cross_section:,pointer_children+1] = partB1#Bottom part of parent 1
                pointer_children = pointer_children + 2
        i = i +1
    return df_offspring

In [15]:
rent = parents(population,0)
rent

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,1.0,1.0
5,0.0,0.0,1.0,1.0,1.0,1.0,1.0
6,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [16]:
input1 = offsprings(rent)
input1

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [17]:
parentals = parent_bins(population)
display(parentals)
children = offsprings(parentals)
display(children)

,list0,list1,list2,list3,list4,list5,list6
0,0.0,NaN,2.0,0.0,2.0,NaN,2.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,NaN,0.0,NaN,NaN,NaN,0.0,NaN
3,0.0,NaN,0.0,1.0,1.0,2.0,0.0
4,0.0,1.0,1.0,2.0,0.0,0.0,0.0
5,2.0,1.0,0.0,0.0,0.0,0.0,0.0
6,2.0,2.0,0.0,2.0,0.0,NaN,1.0


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,NaN,0.0,2.0,0.0,0.0,0.0,2.0,0.0,NaN,...,0.0,NaN,0.0,2.0,2.0,NaN,2.0,2.0,NaN,2.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
3,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,...,2.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0
4,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,2.0,0.0,2.0,2.0,2.0,0.0,2.0,NaN,2.0,...,NaN,2.0,1.0,2.0,NaN,0.0,1.0,0.0,1.0,NaN


 ## 3. Mutation
 Make a random bit swop in each of the population of children of bins 
 1. Select a random position in the population bins
 2. make a bit swap ie. if set to true then set to false, vice-versa
 

In [18]:
##MUTATION: make a random swap at some rand loc of the list of items corresposing to bins to which they are allocated 
#INPUT: children dataframe - has correspoding bin location to items
#OUTPUT: mutated children dataframe
#method, for a random item of the list, randomly change the bin to which it is allocated. 
def mutation(df_children,df_vehicles):
    num_bins = df_vehicles.shape[0]#number of bins
    num_items = df_children.shape[0]#number of items
    num_children = df_children.shape[1]#number of children
    rand_mutation_location = randint(0, num_items)#random location: row location, between 0 & number of items (selects random row)
    df_mutation = pd.DataFrame(np.random.randint(0,num_bins,size=(1, num_children)))#generates a single row of a dataframe, which has random bins. This will be used to swap a row in the children population
    df_children.iloc[rand_mutation_location,:] = df_mutation.iloc[0,:]#make replacemnet at the location
    df_mutated_children = df_children
#     print(rand_mutation_location)
    return df_mutated_children

In [19]:
df = pd.DataFrame(np.random.randint(0,2,size=(1, 42)))
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1,1,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,1,0


In [20]:
display(children)
mutation_kids = mutation(children,df_vehicles1)
display(mutation_kids)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,NaN,0.0,2.0,0.0,0.0,0.0,2.0,0.0,NaN,...,0.0,NaN,0.0,2.0,2.0,NaN,2.0,2.0,NaN,2.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
3,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,...,2.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0
4,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,2.0,0.0,2.0,2.0,2.0,0.0,2.0,NaN,2.0,...,NaN,2.0,1.0,2.0,NaN,0.0,1.0,0.0,1.0,NaN


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,NaN,0.0,2.0,0.0,0.0,0.0,2.0,0.0,NaN,...,0.0,NaN,0.0,2.0,2.0,NaN,2.0,2.0,NaN,2.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
3,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,...,2.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0
4,0.0,0.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,2.0,...,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,2.0,1.0
5,1.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,2.0,0.0,2.0,2.0,2.0,0.0,2.0,NaN,2.0,...,NaN,2.0,1.0,2.0,NaN,0.0,1.0,0.0,1.0,NaN


In [21]:
# temp = mutation_kids.iloc[:,0]
# temp = pd.DataFrame(mutation_kids.loc[:,0])
# k = temp.groupby(0)[0]
# display(k)
temp = mutation_kids.loc[mutation_kids[2] == 2]
display(temp)
ind = list(temp.index)
len(ind)
# print(type(temp))
# list(mutation_kids)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41


0

## 4. Fitness
Rank all bins according to the following criteria: 
1. Number of bins used
2. Number of items not meeting delivery time
3. Cost of delivery

#### Bin refinements
Pseudocode:
1. For each child (column) of the mutated children dataframe
2. For each bin that is available
3. Groupby the bin name
( Sort these items / not sort them as randomly the fitted) 
4. Apply fit algorithm (once bin filled, item allocated to NaN) 
(remainder items run fit algorithm) 

In [22]:
#check if bins have more items allocated to them than there capacity (overfilled)
def overfilled(df_overfilled, df_mutated, id_child, df_items, df_vehicles, dimension, id_bin,items):
    #INPUTS: df_overfilled datafame: is a dataframe which has all the bins as rows and all the children as colms. setting boolean value to true if overfilled. 
            #id_child: col that we looking at in df_mutated
            #id_bin - bin number that we considering
            #df_mutated_bin - dataframe with the selected bin for id_child
            #items - list of items that we considering
           
            
    #1 Dimensional case:
 
    if dimension == 1:
        #total space required for items that have been allocated to bin id_child:
        temp_items = df_items.loc[items]#list of items that are considered 
        total_item_space_required = temp_items.loc[:,'size'].sum()#total space required for the items 
        #total space of the bin:
        bin_size = df_vehicles.loc[id_bin,'size']
        #check if overfilled:
        if total_item_space_required > bin_size:
            df_overfilled.iloc[id_bin,id_child] = True #set the bin to be overfilled
        else:
            df_overfilled.iloc[id_bin,id_child] = False #set the bin not be overfilled
        
    #2 Dimensional case: 
    
    elif dimension == 2:
        #total space required for items that have been allocated to bin id_child:
        temp_items = df_mutated.loc[items]#list of items that are considered 
        total_item_space_required_x = temp_items.loc[:,'x_size'].sum()#total space (x-dimension) required for the items 
        total_item_space_required_y = temp_items.loc[:,'y_size'].sum()#total space (y-dimension) required for the items 

        #total space of the bin:
        bin_size_x = df_vehicle.loc[id_bin,'x_size']
        bin_size_y = df_vehicle.loc[id_bin,'ysize']
        #check if overfilled:
        if ((total_item_space_required_x > bin_size_x or total_item_space_required_y > bin_size_y) or (total_item_space_required_x > bin_size_y or total_item_space_required_y > bin_size_x)):
            df_overfilled.iloc[id_bin,id_child] = True #set the bin to be overfilled
        else:
            df_overfilled.iloc[id_bin,id_child] = False #set the bin not be overfilled
        
    return df_overfilled

<font color='red'>NOTE: Must fix the bin usuage. check if the bin is overfilled. 

In [23]:
#INPUT: Mutated children, vehicles, item  dataframes
#OUTPUT: Refined mutated children dataframe

def bin_overfilled_used(df_mutated_children,df_items, df_vehicles, dimension):
    num_bins = df_vehicles.shape[0]
    num_items = df_mutated_children.shape[0]
    num_children = df_mutated_children.shape[1]
    df_overfilled = pd.DataFrame(np.zeros((num_bins,num_children)))#initialises a boolean dataframe  representing overfilled bins
    df_bin_usuage = pd.DataFrame(np.zeros((num_bins,num_children)))#initialises a boolean dataframe  representing used bins
    for i in range(num_children):#for each column of mutated childrem
        for j in range(num_bins):#for each bin 
            df_temp_mutated = df_mutated_children.loc[df_mutated_children[i] == j]#dataframe displays the column(i) which is equal to the vehicle(j)
            list_items = list(df_temp_mutated.index)#return a list of items that we considering
            num_items = len(list_items)#length of list items
            #if no items in the bin
            if num_items == 0:
                df_bin_usuage.iloc[j,i] = False
                df_overfilled.iloc[j,i] = False

            #if there are iems in the bin, check usuage and if overfilled
            elif num_items!=0:
                #bin used (NOTE: MUST FIX, as check if any items actually fit in the bin, else not used)
                df_bin_usuage.iloc[j,i] = True
                #check if the bin is overfilled
                id_child = i
                id_bin = j
                items = list_items
                if dimension == 1:
                    dimension = 1
                    df_overfilled = overfilled(df_overfilled, df_temp_mutated, id_child, df_items, df_vehicles, dimension, id_bin,items)
                elif dimension == 2:
                    dimension = 2
                    df_overfilled = overfilled(df_overfilled, df_temp_mutated, id_child, df_items, df_vehicles, dimension, id_bin,items)
    return(df_overfilled, df_bin_usuage)
    

## FITNESS
1. Number of used bins
2. Number of overfilled bins
3. Cost of delivery
4. Number of late deliveries
<font color='red'>NOTE: TO complete fitness

In [24]:
def fitness(df_mutated, df_items, df_vehicles, df_overfilled, df_bin_usage):#df_distance_matrix,
    num_items = df_items.shape[0]
    num_bins = df_vehicles.shape[0]
    num_children = df_mutated.shape[1]
    df_fitness = pd.DataFrame(np.zeros((4,num_children)))#NOTE: 4 = number of fitness characteristics that we are considering
    df_ranking = pd.DataFrame(np.zeros((num_children,4)))
    df_ranking.columns = ['rank_used_bins', 'rank_overfilled_bins', 'rank_cost_delivery', 'rank_late_deliveries']
    #NOTE: 
        #row0-row3 = fitness
    #row0: number of bins used
    df_fitness.iloc[0,:] = df_bin_usage.iloc[:,:].sum()
    #row1: number of overfilled bins
    df_fitness.iloc[1,:] = df_overfilled.iloc[:,:].sum()
    #row2: cost of delivery
    #row3: number of late deliveries
    
    #transpose the matrix to index by child
    df_fitness = df_fitness.transpose()
    df_fitness.columns = ['num_used_bins', 'num_overfilled_bins', 'cost_delivery', 'num_late_deliveries']
    cols = list(df_fitness)
    rank_cols = list(df_ranking)
    for i in range(4): #for the number of fitness tests
        col_name = cols[i]
        rank_col_name = rank_cols[i]
        df_fitness_copy = df_fitness.copy()
        df_fitness_copy = df_fitness_copy.sort_values(by=['%s'%col_name], axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
        df_ranking.loc[:,'%s'%rank_col_name] = list(df_fitness_copy.index)
    return df_fitness, df_ranking

In [25]:
ovf, binU = bin_overfilled_used(mutation_kids,df_items1, df_vehicles1, 1)
df_fit, df_rank = fitness(mutation_kids, df_items1, df_vehicles1, ovf, binU)
display(df_fit)
display(df_rank)

,num_used_bins,num_overfilled_bins,cost_delivery,num_late_deliveries
0,3.0,1.0,0.0,0.0
1,2.0,1.0,0.0,0.0
2,2.0,1.0,0.0,0.0
3,2.0,1.0,0.0,0.0
4,3.0,1.0,0.0,0.0
5,3.0,1.0,0.0,0.0
6,2.0,1.0,0.0,0.0
7,3.0,1.0,0.0,0.0
8,3.0,0.0,0.0,0.0
9,3.0,1.0,0.0,0.0


,rank_used_bins,rank_overfilled_bins,rank_cost_delivery,rank_late_deliveries
0,23,15,0,0
1,20,25,23,23
2,29,32,24,24
3,30,8,25,25
4,18,28,26,26
5,16,0,27,27
6,34,23,28,28
7,11,24,29,29
8,10,26,30,30
9,12,27,31,31
